# Multivariate linear regression using stochastic gradient descent

So we will be implementing a multivariate linear regresssion model using stochastic gradient descent.
Gradient descent in ML is the process of minimizing a function following the slope or gradient of that function.

### How does it work?
The way this optimization algorithm works is that each training instance is shown to the
model one at a time. The model makes a prediction for a training instance, the error is calculated
and the model is updated in order to reduce the error for the next prediction. This process is
repeated for a fixed number of iterations.

# Steps in this tutorial

1. Make predictions
2. Estimate coefficients
3. Case study on [Wine dataset](https://www.kaggle.com/datasets/rajyellow46/wine-quality)

## Making predictions function

In [17]:
def prediction(row, coefficients):
    yhat = coefficients[0]
    
    for i in range(len(row) - 1):
        yhat += coefficients[i+1] * row[i]
    return yhat

## Testing this on a dummy example

In [18]:
dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
coef = [0.4, 0.8]
for row in dataset:
    yhat = prediction(row, coef)
    print(f"Expected: {row[-1]}, Predicted: {round(yhat, 3)}")

Expected: 1, Predicted: 1.2
Expected: 3, Predicted: 2.0
Expected: 3, Predicted: 3.6
Expected: 2, Predicted: 2.8
Expected: 5, Predicted: 4.4


## Estimating coefficients

Two important terms to keep in mind:

1. Learning rate
Used to limit the amount that each coefficient is corrected each time it
is updated.
2. Epochs:
The number of times to run through the training data while updating the
coefficients

In [21]:
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            yhat = prediction(row, coef)
            error = yhat - row[-1]
            sum_error += (error**2)
            
            coef[0] = coef[0] - l_rate * error
            
            for i in range (len(row) - 1):
                coef[i+1] = coef[i+1] - l_rate * error * row[i]
        print(f"epoch: {epoch}; lrate: {round(l_rate, 3)}; error: {round(sum_error, 3)}")
    return coef

## Testing on dummy dataset

In [29]:
dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
l_rate = 0.001
epoch = 10

coef = coefficients_sgd(dataset, l_rate, epoch)
print(coef)


epoch: 0; lrate: 0.001; error: 46.236
epoch: 1; lrate: 0.001; error: 41.305
epoch: 2; lrate: 0.001; error: 36.93
epoch: 3; lrate: 0.001; error: 33.047
epoch: 4; lrate: 0.001; error: 29.601
epoch: 5; lrate: 0.001; error: 26.543
epoch: 6; lrate: 0.001; error: 23.83
epoch: 7; lrate: 0.001; error: 21.422
epoch: 8; lrate: 0.001; error: 19.285
epoch: 9; lrate: 0.001; error: 17.389
[0.10710331074898374, 0.3801081881174074]


# Wine case study

## Dataset preparation functions

In [58]:
# Importing necessary libraries

from random import seed
from random import randrange
from csv import reader
from math import sqrt

# Load csv function
def load_csv_skip(filename):
    dataset = list()
    with open(filename, "r") as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
        return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
        
# pop the first row 
def pop_first_row(dataset):
    dataset.pop(0)
    

## Normalization, cross-validation split, RMSE

In [59]:
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax

# Normalizing the data
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


# Calculate root mean squared error
def rmse_metric(actual, predicted):
	sum_error = 0.0
	for i in range(len(actual)):
		prediction_error = predicted[i] - actual[i]
		sum_error += (prediction_error ** 2)
	mean_error = sum_error / float(len(actual))
	return sqrt(mean_error)

## Evaluation

In [60]:
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		rmse = rmse_metric(actual, predicted)
		scores.append(rmse)
	return scores

## Linear regression with gradient descent

In [67]:
def linear_regression_sgd(train, test, l_rate, n_epoch):
	predictions = list()
	coef = coefficients_sgd(train, l_rate, n_epoch)
	for row in test:
		yhat = prediction(row, coef)
		predictions.append(yhat)
	return(predictions)

## Multivariate linear regression functions

In [68]:
seed(1)
# load and prepare data
filename = 'wine.csv'
dataset = load_csv(filename)

for i in range(len(dataset[0])):
	str_column_to_float(dataset, i)
# normalize
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.01
n_epoch = 50
scores = evaluate_algorithm(dataset, linear_regression_sgd, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean RMSE: %.3f' % (sum(scores)/float(len(scores))))

epoch: 0; lrate: 0.01; error: 95.749
epoch: 1; lrate: 0.01; error: 81.046
epoch: 2; lrate: 0.01; error: 79.556
epoch: 3; lrate: 0.01; error: 78.795
epoch: 4; lrate: 0.01; error: 78.318
epoch: 5; lrate: 0.01; error: 78.004
epoch: 6; lrate: 0.01; error: 77.793
epoch: 7; lrate: 0.01; error: 77.647
epoch: 8; lrate: 0.01; error: 77.545
epoch: 9; lrate: 0.01; error: 77.472
epoch: 10; lrate: 0.01; error: 77.418
epoch: 11; lrate: 0.01; error: 77.378
epoch: 12; lrate: 0.01; error: 77.348
epoch: 13; lrate: 0.01; error: 77.324
epoch: 14; lrate: 0.01; error: 77.305
epoch: 15; lrate: 0.01; error: 77.29
epoch: 16; lrate: 0.01; error: 77.277
epoch: 17; lrate: 0.01; error: 77.266
epoch: 18; lrate: 0.01; error: 77.257
epoch: 19; lrate: 0.01; error: 77.249
epoch: 20; lrate: 0.01; error: 77.242
epoch: 21; lrate: 0.01; error: 77.235
epoch: 22; lrate: 0.01; error: 77.23
epoch: 23; lrate: 0.01; error: 77.224
epoch: 24; lrate: 0.01; error: 77.22
epoch: 25; lrate: 0.01; error: 77.215
epoch: 26; lrate: 0.01; e